In [1]:
import os
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
import warnings
import glob 
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Desactiva las advertencias
warnings.filterwarnings('ignore')

In [2]:
# Definir las columnas de códigos y de diagnósticos en texto libre
columnas_codigos = ['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3',
                    'COD_DIAGNOSTICO4', 'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6',
                    'COD_DIAGNOSTICO7', 'COD_DIAGNOSTICO8', 'COD_DIAGNOSTICO9',
                    'COD_DIAGNOSTICO10']
columnas_diagnosticos = ['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3',
                         'DIAGNOSTICO4', 'DIAGNOSTICO5', 'DIAGNOSTICO6',
                         'DIAGNOSTICO7', 'DIAGNOSTICO8', 'DIAGNOSTICO9',
                         'DIAGNOSTICO10']


In [3]:
def cargar_dataframe_ambulatorio(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def cargar_dataframe_ieeh(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def filtro_dataframe_ca_tiroides(dataframe):

    # Patrón para identificar los códigos CIE-10 asociados al cáncer gástrico
    patron_cancer_gastrico = r'^C16'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)


    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer gástrico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_gastrico, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer gástrico en texto libre
    patron_adenocarcinoma_gastrico = re.compile(r'adenocarcinoma.*g[aá]strico', re.IGNORECASE)
    patron_carcinoma_gastrico = re.compile(r'carcinoma.*g[aá]strico', re.IGNORECASE)
    #patron_linfoma_gastrico = re.compile(r'linfoma.*g[aá]strico', re.IGNORECASE)
    #patron_sarcoma_gastrico = re.compile(r'sarcoma.*g[aá]strico', re.IGNORECASE)
    patron_neoplasia_gastrica = re.compile(r'neoplasia.*g[aá]strica', re.IGNORECASE)
    patron_tumor_estomago = re.compile(r'tumor.*est[oó]mago', re.IGNORECASE)
    patron_cancer_estomago = re.compile(r'c[aá]ncer.*est[oó]mago', re.IGNORECASE)
    #patron_adenocarcinoma_intestinal = re.compile(r'adenocarcinoma.*intestinal', re.IGNORECASE)
    #patron_adenocarcinoma_difuso = re.compile(r'adenocarcinoma.*difuso', re.IGNORECASE)
    #patron_carcinoma_anillo_sello = re.compile(r'carcinoma.*anillo.*sello', re.IGNORECASE)
    patron_cancer_gastrico_metastasico = re.compile(r'c[aá]ncer.*g[aá]strico.*metast[aá]sico', re.IGNORECASE)
    #patron_tumor_gist_gastrico = re.compile(r'tumor.*gist.*g[aá]strico', re.IGNORECASE)
    patron_gastrico = re.compile(r'\bc[aá](?!o)[a-zá-ú]*[ \.]*g[aá]strico(?!o)[a-zá-ú]*\b', re.IGNORECASE)

    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_adenocarcinoma_gastrico, na=False) |
        x.str.contains(patron_carcinoma_gastrico, na=False) |
        x.str.contains(patron_gastrico, na=False) |
        #x.str.contains(patron_sarcoma_gastrico, na=False) |
        x.str.contains(patron_neoplasia_gastrica, na=False) |
        x.str.contains(patron_tumor_estomago, na=False) |
        x.str.contains(patron_cancer_estomago, na=False) |
        #x.str.contains(patron_adenocarcinoma_intestinal, na=False) |
        #x.str.contains(patron_adenocarcinoma_difuso, na=False) |
        #x.str.contains(patron_carcinoma_anillo_sello, na=False) |
        x.str.contains(patron_cancer_gastrico_metastasico, na=False)
        #x.str.contains(patron_tumor_gist_gastrico, na=False)
        ).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[filtro_codigos | filtro_diagnosticos].copy()


def filtro_dataframe_ca_tiroides_ieeh(dataframe):

    # Definir las columnas de códigos y de diagnósticos en texto libre

    columnas_diagnosticos = ['DIAG1','DIAG2','DIAG3']

    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    # Llenar los valores NaN con una cadena vacía en las columnas de diagnóstico
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].fillna('')


    # Patrones para identificar diagnósticos de cáncer gástrico en texto libre
    patron_adenocarcinoma_gastrico = re.compile(r'adenocarcinoma.*g[aá]strico', re.IGNORECASE)
    patron_carcinoma_gastrico = re.compile(r'carcinoma.*g[aá]strico', re.IGNORECASE)
    #patron_linfoma_gastrico = re.compile(r'linfoma.*g[aá]strico', re.IGNORECASE)
    #patron_sarcoma_gastrico = re.compile(r'sarcoma.*g[aá]strico', re.IGNORECASE)
    patron_neoplasia_gastrica = re.compile(r'neoplasia.*g[aá]strica', re.IGNORECASE)
    patron_tumor_estomago = re.compile(r'tumor.*est[oó]mago', re.IGNORECASE)
    patron_cancer_estomago = re.compile(r'c[aá]ncer.*est[oó]mago', re.IGNORECASE)
    #patron_adenocarcinoma_intestinal = re.compile(r'adenocarcinoma.*intestinal', re.IGNORECASE)
    #patron_adenocarcinoma_difuso = re.compile(r'adenocarcinoma.*difuso', re.IGNORECASE)
    #patron_carcinoma_anillo_sello = re.compile(r'carcinoma.*anillo.*sello', re.IGNORECASE)
    patron_cancer_gastrico_metastasico = re.compile(r'c[aá]ncer.*g[aá]strico.*metast[aá]sico', re.IGNORECASE)
    #patron_tumor_gist_gastrico = re.compile(r'tumor.*gist.*g[aá]strico', re.IGNORECASE)
    patron_gastrico = re.compile(r'\bc[aá](?!o)[a-zá-ú]*[ \.]*g[aá]strico(?!o)[a-zá-ú]*\b', re.IGNORECASE)

    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_adenocarcinoma_gastrico, na=False) |
        x.str.contains(patron_carcinoma_gastrico, na=False) |
        x.str.contains(patron_gastrico, na=False) |
        #x.str.contains(patron_sarcoma_gastrico, na=False) |
        x.str.contains(patron_neoplasia_gastrica, na=False) |
        x.str.contains(patron_tumor_estomago, na=False) |
        x.str.contains(patron_cancer_estomago, na=False) |
        #x.str.contains(patron_adenocarcinoma_intestinal, na=False) |
        #x.str.contains(patron_adenocarcinoma_difuso, na=False) |
        #x.str.contains(patron_carcinoma_anillo_sello, na=False) |
        x.str.contains(patron_cancer_gastrico_metastasico, na=False)
        #x.str.contains(patron_tumor_gist_gastrico, na=False)
        ).any(axis=1)


    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[filtro_diagnosticos].copy()

def filtro_dataframe_ca_tiroides_urgencias(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['CODDIAGNOSTICO']
    columnas_diagnosticos = ['DESCDIAGNOSTICO']

    # Patrón para identificar los códigos CIE-10 asociados al cáncer gástrico
    patron_cancer_gastrico = r'^C16'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer gástrico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_gastrico, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer gástrico en texto libre
    patron_adenocarcinoma_gastrico = re.compile(r'adenocarcinoma.*g[aá]strico', re.IGNORECASE)
    patron_carcinoma_gastrico = re.compile(r'carcinoma.*g[aá]strico', re.IGNORECASE)
    #patron_linfoma_gastrico = re.compile(r'linfoma.*g[aá]strico', re.IGNORECASE)
    #patron_sarcoma_gastrico = re.compile(r'sarcoma.*g[aá]strico', re.IGNORECASE)
    patron_neoplasia_gastrica = re.compile(r'neoplasia.*g[aá]strica', re.IGNORECASE)
    patron_tumor_estomago = re.compile(r'tumor.*est[oó]mago', re.IGNORECASE)
    patron_cancer_estomago = re.compile(r'c[aá]ncer.*est[oó]mago', re.IGNORECASE)
    #patron_adenocarcinoma_intestinal = re.compile(r'adenocarcinoma.*intestinal', re.IGNORECASE)
    #patron_adenocarcinoma_difuso = re.compile(r'adenocarcinoma.*difuso', re.IGNORECASE)
    #patron_carcinoma_anillo_sello = re.compile(r'carcinoma.*anillo.*sello', re.IGNORECASE)
    patron_cancer_gastrico_metastasico = re.compile(r'c[aá]ncer.*g[aá]strico.*metast[aá]sico', re.IGNORECASE)
    #patron_tumor_gist_gastrico = re.compile(r'tumor.*gist.*g[aá]strico', re.IGNORECASE)
    patron_gastrico = re.compile(r'\bc[aá](?!o)[a-zá-ú]*[ \.]*g[aá]strico(?!o)[a-zá-ú]*\b', re.IGNORECASE)

    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_adenocarcinoma_gastrico, na=False) |
        x.str.contains(patron_carcinoma_gastrico, na=False) |
        x.str.contains(patron_gastrico, na=False) |
        #x.str.contains(patron_sarcoma_gastrico, na=False) |
        x.str.contains(patron_neoplasia_gastrica, na=False) |
        x.str.contains(patron_tumor_estomago, na=False) |
        x.str.contains(patron_cancer_estomago, na=False) |
        #x.str.contains(patron_adenocarcinoma_intestinal, na=False) |
        #x.str.contains(patron_adenocarcinoma_difuso, na=False) |
        #x.str.contains(patron_carcinoma_anillo_sello, na=False) |
        x.str.contains(patron_cancer_gastrico_metastasico, na=False)
        #x.str.contains(patron_tumor_gist_gastrico, na=False)
        ).any(axis=1)


    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[filtro_codigos | filtro_diagnosticos].copy()

def filtro_dataframe_ca_tiroides_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Patrón para identificar los códigos CIE-10 asociados al cáncer gástrico
    patron_cancer_gastrico = r'^C16'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)

    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer gástrico
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_gastrico, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer gástrico en texto libre
    patron_adenocarcinoma_gastrico = re.compile(r'adenocarcinoma.*g[aá]strico', re.IGNORECASE)
    patron_carcinoma_gastrico = re.compile(r'carcinoma.*g[aá]strico', re.IGNORECASE)
    #patron_linfoma_gastrico = re.compile(r'linfoma.*g[aá]strico', re.IGNORECASE)
    #patron_sarcoma_gastrico = re.compile(r'sarcoma.*g[aá]strico', re.IGNORECASE)
    patron_neoplasia_gastrica = re.compile(r'neoplasia.*g[aá]strica', re.IGNORECASE)
    patron_tumor_estomago = re.compile(r'tumor.*est[oó]mago', re.IGNORECASE)
    patron_cancer_estomago = re.compile(r'c[aá]ncer.*est[oó]mago', re.IGNORECASE)
    #patron_adenocarcinoma_intestinal = re.compile(r'adenocarcinoma.*intestinal', re.IGNORECASE)
    #patron_adenocarcinoma_difuso = re.compile(r'adenocarcinoma.*difuso', re.IGNORECASE)
    #patron_carcinoma_anillo_sello = re.compile(r'carcinoma.*anillo.*sello', re.IGNORECASE)
    patron_cancer_gastrico_metastasico = re.compile(r'c[aá]ncer.*g[aá]strico.*metast[aá]sico', re.IGNORECASE)
    #patron_tumor_gist_gastrico = re.compile(r'tumor.*gist.*g[aá]strico', re.IGNORECASE)
    patron_gastrico = re.compile(r'\bc[aá](?!o)[a-zá-ú]*[ \.]*g[aá]strico(?!o)[a-zá-ú]*\b', re.IGNORECASE)

    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_adenocarcinoma_gastrico, na=False) |
        x.str.contains(patron_carcinoma_gastrico, na=False) |
        x.str.contains(patron_gastrico, na=False) |
        #x.str.contains(patron_sarcoma_gastrico, na=False) |
        x.str.contains(patron_neoplasia_gastrica, na=False) |
        x.str.contains(patron_tumor_estomago, na=False) |
        x.str.contains(patron_cancer_estomago, na=False) |
        #x.str.contains(patron_adenocarcinoma_intestinal, na=False) |
        #x.str.contains(patron_adenocarcinoma_difuso, na=False) |
        #x.str.contains(patron_carcinoma_anillo_sello, na=False) |
        x.str.contains(patron_cancer_gastrico_metastasico, na=False)
        #x.str.contains(patron_tumor_gist_gastrico, na=False)
        ).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[(filtro_codigos | filtro_diagnosticos) & filtro_comuna].copy()
    

def agregar_dv(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    #condicion = (resultado_filtrado['TIPO_IDENTIFICADOR_PACIENTE'] == 'RUN')
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[-1]

    #resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
    resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]

    return resultado_filtrado

def agregar_dv_urgencias(resultado_filtrado):

    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['RUT'].str[-1]

    resultado_filtrado['RUT'] = resultado_filtrado['RUT'].str[:-1]

    return resultado_filtrado

def agregar_dv_inter(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE

    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[-1]

    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[:-1]

    return resultado_filtrado

def obtener_diagnosticos_y_codigos(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR_PACIENTE'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4',
                                'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7','DIAGNOSTICO8',
                                'DIAGNOSTICO9','DIAGNOSTICO10']].values.flatten()
    codigos_diagnostico = df_paciente[['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3', 'COD_DIAGNOSTICO4',
                                       'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6', 'COD_DIAGNOSTICO7','COD_DIAGNOSTICO8',
                                       'COD_DIAGNOSTICO9','COD_DIAGNOSTICO10']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_ieeh(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAG1','DIAG2','DIAG3']].values.flatten()
    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]

    return list(set(diagnosticos))

def obtener_diagnosticos_urgencias(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DESCDIAGNOSTICO']].values.flatten()
    codigos_diagnostico = df_paciente[['CODDIAGNOSTICO']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_inter(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR'] == nro_identificacion]
    diagnosticos = df_paciente[['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']].values.flatten()
    codigos_diagnostico = df_paciente[['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def agregar_diagnosticos_y_codigos(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR_PACIENTE').agg({'NRO_IDENTIFICADOR_PACIENTE': 'first', 'DIAGNOSTICO1': 'first',
                                                            'DIAGNOSTICO2': 'first','DIAGNOSTICO3': 'first','DIAGNOSTICO4': 'first',
                                                            'DIAGNOSTICO5': 'first','DIAGNOSTICO6': 'first','DIAGNOSTICO7': 'first',
                                                            'DIAGNOSTICO8': 'first','DIAGNOSTICO9': 'first','DIAGNOSTICO10': 'first',
                                                            'COD_DIAGNOSTICO1': 'first','COD_DIAGNOSTICO2': 'first', 'COD_DIAGNOSTICO2': 'first',
                                                            'COD_DIAGNOSTICO3': 'first', 'COD_DIAGNOSTICO4': 'first', 'COD_DIAGNOSTICO5': 'first',
                                                            'COD_DIAGNOSTICO6': 'first', 'COD_DIAGNOSTICO7': 'first','COD_DIAGNOSTICO8': 'first',
                                                                'COD_DIAGNOSTICO8': 'first',  'COD_DIAGNOSTICO9': 'first',
                                                                'COD_DIAGNOSTICO10': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR_PACIENTE'].apply(lambda x: obtener_diagnosticos_y_codigos(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR_PACIENTE': 'RUT'})
    return resultados

def agregar_diagnosticos_ieeh(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])
    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DIAG1': 'first',
                                                            'DIAG1': 'first','DIAG3': 'first'}).reset_index(drop=True)
    #resultados['CODIGOS_DIAGNOSTICO'] = []
    resultados['DIAGNOSTICOS'] = resultados['RUT'].apply(lambda x: obtener_diagnosticos_ieeh(resultado_filtrado, x))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})
    resultados['CODIGOS_DIAGNOSTICO'] = np.nan
    return resultados

def agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DESCDIAGNOSTICO': 'first',
                                                        'CODDIAGNOSTICO': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['RUT'].apply(lambda x: obtener_diagnosticos_urgencias(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})

    return resultados

def agregar_diagnosticos_y_codigos_inter(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR').agg({'NRO_IDENTIFICADOR': 'first', 'SOSP_DIAG_DESC_1': 'first',
                                                            'SOSP_DIAG_DESC_2': 'first','SOSP_DIAG_DESC_3': 'first','SOSP_DIAG_DESC_4': 'first',
                                                            'SOSP_DIAG_DESC_5': 'first','SOSP_DIAG_COD_1': 'first','SOSP_DIAG_COD_2': 'first',
                                                                      'SOSP_DIAG_COD_3': 'first',
                                                            'SOSP_DIAG_COD_4': 'first', 'SOSP_DIAG_COD_5': 'first'
                                                            }).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR'].apply(lambda x: obtener_diagnosticos_inter(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR': 'RUT'})
    return resultados

def crear_dataframe_ambulatorio(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer Gástrico',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['PACIENTE'],
        
        'N° Telefono 1': resultado_filtrado.get('TELEFONO_MOVIL', np.nan),
        'N° Telefono 2':resultado_filtrado.get('TELEFONO_FIJO_1', np.nan),
        'N° Telefono 3': resultado_filtrado.get('TELEFONO_FIJO_2', np.nan),
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
   
    return nuevo_dataframe

def crear_dataframe_ieeh(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer Gástrico',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES']+ ' ' + resultado_filtrado['APELL_PATE']+ ' '+ resultado_filtrado['APELL_MATE'] ,
        'N° Telefono 1': resultado_filtrado['TELEFONO'],
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe


def crear_dataframe_urgencias(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer Gástrico',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente': resultado_filtrado['NOMBRES_PACIENTE']+ ' '+ resultado_filtrado['APELLIDOS_PACIENTE'],
        'N° Telefono 1': np.nan,
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_inter(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer Gástrico',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOM_PAC'] + ' ' + resultado_filtrado['APELLPAT_PAC']+' '+resultado_filtrado['APELLMAT_PAC'],
        'N° Telefono 1': resultado_filtrado['FONO_MOVIL'],
        'N° Telefono 2': resultado_filtrado['FONO_FIJO_1'],
        'N° Telefono 3': resultado_filtrado['FONO_FIJO_2'],
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def safe_convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return value

def merge_ambulatorio(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.
    
    return merged_df 

def merge_urgencia(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)


    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def merge_inter(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)


    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def filtrar_columnas_relevantes(merged_df):
    nuevo_dataframe = merged_df.drop_duplicates(subset='RUT', keep='first')
    nuevo_dataframe = nuevo_dataframe[['Problema de Salud','RUT' ,'DV',
       'Nombre del Paciente', 'N° Telefono 1', 'N° Telefono 2',
       'N° Telefono 3', 'Fecha Proximo Hito', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO',
       'Nombre agenda Medico especialista']]
    nuevo_dataframe = nuevo_dataframe.rename(columns={'Problema de Salud': 'Problema Oncologico'})
    nuevo_dataframe['Nombre garantia'] = np.nan
    nuevo_dataframe= nuevo_dataframe[['Problema Oncologico',
    'RUT',
    'DV',
    'Nombre del Paciente',
    'N° Telefono 1',
    'N° Telefono 2',
    'N° Telefono 3',
    'Nombre garantia',
    'Fecha Proximo Hito',
    'DIAGNOSTICOS',
    'CODIGOS_DIAGNOSTICO',
    'Nombre agenda Medico especialista']]
    return nuevo_dataframe

def crear_dataframe_ges(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'Problema Oncologico': dataframe_tiroides['Problema de salud'],
        'RUT': dataframe_tiroides['RUT'],
        'DV': dataframe_tiroides['DV'],
        'Nombre del Paciente':dataframe_tiroides['Nombre'],
        'N° Telefono 1': dataframe_tiroides['Telefonos.x'],
        'N° Telefono 2': dataframe_tiroides['Telefonos.y'],
        'N° Telefono 3': np.nan,
        'Nombre garantia': dataframe_tiroides['Nombre garantia'],
        'Fecha Proximo Hito': np.nan,
        'DIAGNOSTICOS': np.nan,
        'CODIGOS_DIAGNOSTICO': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
    return nuevo_dataframe_2

def crear_dataframe_ultima_atencion(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'RUT': dataframe_tiroides['RUT'],
        'Fecha Ultima atencion': dataframe_tiroides['HORA_TERMINO_ATENCION'],
        'Profesional Ultima atencion': dataframe_tiroides['ESTAMENTO_PROFESIONAL_ATENCION']+ ' '+
        dataframe_tiroides['NOMBRE_PROFESIONAL_ATENCION']+ ' ' + dataframe_tiroides['DESCRIPCION'] + ' '+
        dataframe_tiroides['ESPECIALIDAD_AGENDA']
    })
    return nuevo_dataframe_2

def proceso_dataframe(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
        resultado_filtrado = agregar_dv(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales 

def proceso_dataframe_ultimo(ruta_archivo):
    dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
    resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
    resultado_filtrado = agregar_dv(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
    merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
    return nuevo_dataframe

def proceso_dataframe_ieeh(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_ieeh(dataframe)
        resultados = agregar_diagnosticos_ieeh(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ieeh(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_urgencias(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_urgencias(dataframe)
        resultado_filtrado = agregar_dv_urgencias(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_urgencias(resultado_filtrado)
        merged_df = merge_urgencia(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_inter(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada'])]
    dataframe = dataframe[~dataframe['NRO_IDENTIFICADOR'].str.contains('AT')]
    # Definir la fecha de inicio para el filtro
    fecha_hoy = datetime.today()
    fecha_inicio = fecha_hoy - timedelta(days=10)

    fecha_inicio = pd.to_datetime(fecha_inicio, format='%d/%m/%Y')
    # Filtrar el DataFrame
    dataframe = dataframe[dataframe['FECHA_SOLIC'] >= fecha_inicio]
    resultado_filtrado = filtro_dataframe_ca_tiroides_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado)
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe



def ultimo_csv_en_carpeta(ruta):
    # Obtener lista de archivos en la carpeta
    archivos = os.listdir(ruta)
    
    # Filtrar archivos que sean .csv
    archivos_csv = [archivo for archivo in archivos if archivo.endswith('.csv')]
    
    if not archivos_csv:
        return None  # Si no hay archivos .csv en la carpeta
    
    # Ordenar la lista de archivos por fecha de modificación
    archivos_csv.sort(key=lambda x: os.path.getmtime(os.path.join(ruta, x)), reverse=True)
    
    # Devolver la ruta completa del último archivo .csv
    return os.path.join(ruta, archivos_csv[0])

def convertir_fecha(fecha):
    try:
        return pd.to_datetime(fecha, format='%d/%m/%Y %H:%M')
    except ValueError:
        return pd.to_datetime(fecha, format='%d-%m-%y %H:%M')
    
def obtener_fechas_amb(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        dataframe['DV'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[-1]
        dataframe['RUT'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
        



        resultados_totales.append(dataframe)
        
    dataframe = pd.concat(resultados_totales)

    
    # Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
    endocrinologia_df = dataframe[dataframe['ESPECIALIDAD_AGENDA'].isin(['Cirugía Abdominal Adulto','Medicina Interna','Imagenología AMB'])]
    endocrinologia_df = endocrinologia_df[endocrinologia_df['ESTAMENTO_PROFESIONAL_ATENCION']=='Médico']

    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
    endocrinologia_df = endocrinologia_df[~endocrinologia_df['ESTADO_CITA'].isin(['No realizada', 'Anulada'])]
    # Convertir la columna 'HORA_TERMINO_ATENCION' a formato de fecha y hora
    #endocrinologia_df['HORA_TERMINO_ATENCION'] = pd.to_datetime(endocrinologia_df['HORA_TERMINO_ATENCION'], format='mixed')
    endocrinologia_df['HORA_TERMINO_ATENCION'] = endocrinologia_df['HORA_TERMINO_ATENCION'].apply(convertir_fecha)

    # Agrupar por RUT y obtener la última fecha de la hora de atención para cada grupo
    #ultima_fecha_por_rut = endocrinologia_df.groupby('RUT').agg({'HORA_TERMINO_ATENCION': 'max'}).reset_index()

    ultima_fecha_por_rut =  endocrinologia_df.groupby('RUT').agg({
    'HORA_TERMINO_ATENCION': 'max',
    'NOMBRE_PROFESIONAL_ATENCION': 'first',
    'ESTAMENTO_PROFESIONAL_ATENCION': 'first',
    'DESCRIPCION': 'first',
    'ESPECIALIDAD_AGENDA': 'first'
}).reset_index()
    # Fusionar el DataFrame original con el DataFrame resultante para recuperar NOMBRE_PROFESIONAL_ATENCION y ESTAMENTO_PROFESIONAL_ATENCION
    resultado = ultima_fecha_por_rut.merge(endocrinologia_df[['RUT', 'HORA_TERMINO_ATENCION', 'NOMBRE_PROFESIONAL_ATENCION', 'ESTAMENTO_PROFESIONAL_ATENCION','DESCRIPCION']], on=['RUT', 'HORA_TERMINO_ATENCION'], how='left')

    return ultima_fecha_por_rut 

# Ruta archivos

In [4]:
dir_actual = os.getcwd()
dir_archivo_csv = os.path.join(dir_actual,'..','Reporteria.lnk')

In [5]:
import win32com.client

def get_target_path(lnk_file):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(lnk_file)
    return shortcut.Targetpath

# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Reporteria.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_reporteria = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_reporteria)


In [6]:
# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Unidad_Gestion_Del_Paciente.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_ges = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_ges)

In [7]:


#################### RUTA ARCHIVOS ##########################################

dias_reportes = 10

# Lista de rutas de archivos
rutas_archivos = []

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)
# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


#Reportes Ambulatorios Histórico
ruta_archivo_4 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', '202401_Reporte_Ambulatorio_Mensual.csv')
ruta_archivo_5 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '20240401_Reporte_Ambulatorio_Diario.csv')


# Lista de rutas de archivos
rutas_archivos_hist = [ruta_archivo_4, ruta_archivo_5]
#agregamos los dias 
for i in range(1,int(day_formatted)+1):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
for i in range(1,today.month):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Ambulatorio', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))

for i in range(1,6):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Ambulatorio', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Anual', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
  



#Reportes Ambulatorios


patron_amb = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')

archivos_amb = glob.glob(patron_amb)

archivos_ordenados_amb = sorted(archivos_amb, key=os.path.getmtime, reverse = True)

rutas_archivos = archivos_ordenados_amb[:dias_reportes]



#Reportes IEEH

rutas_archivos_2 = []


patron_ieeh = os.path.join(ruta_reporteria,'Reporte_IEEH','Diario', '*Informe_Diario-IEEE*')

archivos_ieeh  = glob.glob(patron_ieeh )

archivos_ordenados_ieeh  = sorted(archivos_ieeh , key=os.path.getmtime, reverse = True)

rutas_archivos_2 = archivos_ordenados_ieeh[:dias_reportes]


#Reportes Urgencias


rutas_archivos_3 = []


patron_urg = os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario', '*Urgencias_atenciones*')

archivos_urg  = glob.glob(patron_urg)

archivos_ordenados_urg  = sorted(archivos_urg , key=os.path.getmtime, reverse = True)

rutas_archivos_3 = archivos_ordenados_urg[:dias_reportes]





In [8]:
#Reportes SIC

patron_sic = os.path.join(ruta_reporteria,'Reportes_Solicitudes','Mensual', '*Informe_Solicitudes_DerivacionCE*')

archivos_sic  = glob.glob(patron_sic)

archivos_ordenados_sic  = sorted(archivos_sic , key=os.path.getmtime, reverse = True)

rutas_archivos_4 = archivos_ordenados_sic[:1]


# Generar dataframes

In [9]:
# Llamar a la función proceso_dataframe con la lista de rutas de archivos
resultado = obtener_fechas_amb(rutas_archivos_hist)

resultado_final = proceso_dataframe(rutas_archivos)

resultado_final_2 = proceso_dataframe_ieeh(rutas_archivos_2)

resultado_final_3 = proceso_dataframe_urgencias(rutas_archivos_3)

resultado_final_4 = proceso_dataframe_inter(rutas_archivos_4[0])

In [10]:
#os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')
ruta_del_archivo = os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

In [11]:
# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

# Construir la ruta completa al archivo CSV
dataframe['RUT']= dataframe['RUT'].apply(lambda x : str(x))
dataframe['RUT COMPLETO'] = dataframe['RUT'].astype(str)  + dataframe['DV'].astype(str)
dataframe_tiroides = dataframe[dataframe['Problema de salud'] == 'Cáncer Gástrico . {decreto nº 228}']

dataframe_tiroides["Fecha inicio"]=pd.to_datetime(dataframe_tiroides["Fecha inicio"])
# Ordenar en orden descendente por la columna "FECHA_CITA"
dataframe_tiroides = dataframe_tiroides.sort_values(by='Fecha inicio', ascending=False)


# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"
dataframe_tiroides = dataframe_tiroides.drop_duplicates(subset='RUT', keep='first')

nuevo_dataframe_2 = crear_dataframe_ges(dataframe_tiroides)

resultado_final.append(nuevo_dataframe_2)


#if ultimo_csv:
#    print("La dirección del último archivo .csv en la carpeta es:", ultimo_csv)
    
#    resultado_final.append(proceso_dataframe_ultimo(ultimo_csv))
#    df_concatenated = pd.concat(resultado_final)

#else:
#    print("No se encontraron archivos .csv en la carpeta.")
#    df_concatenated = pd.concat(resultado_final)
 # Concatenar los DataFrames
resultado_final.extend(resultado_final_2)
resultado_final.extend(resultado_final_3)
resultado_final.extend([resultado_final_4])
df_concatenated = pd.concat(resultado_final)






# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"

#consolidated_df = df_concatenated.groupby('RUT').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill')).drop_duplicates().reset_index(drop=True)

#resultado_filtrado_2 = consolidated_df.drop_duplicates(subset='RUT', keep='first')

resultado_filtrado_2 = df_concatenated.drop_duplicates(subset='RUT', keep='first')

# Iterar sobre las filas del primer DataFrame
for index, row in resultado_filtrado_2.iterrows():
    rut = row['RUT']
    
    # Buscar el valor correspondiente en el segundo DataFrame
    nombre_garantia = nuevo_dataframe_2.loc[nuevo_dataframe_2['RUT'] == str(rut), 'Nombre garantia'].values
    
    # Encontrar la fila correspondiente en el primer DataFrame y asignar el valor
    resultado_filtrado_2.loc[resultado_filtrado_2['RUT'] == rut, 'Nombre garantia'] = nombre_garantia[0] if nombre_garantia.size > 0 else None

resultado_filtrado_2['RUT'] = resultado_filtrado_2['RUT'].apply(lambda x: str(x))


In [ ]:
resultado_filtrado_2 = resultado_filtrado_2.drop_duplicates(subset='RUT', keep='first')

In [12]:
#nombre_archivo = 'Total pacientes_nuevos_ca_gastrico.xlsx'  # Nombre del archivo de Excel
#resultado_filtrado_2.to_excel(nombre_archivo, index=False)  # index=False para no incluir el índice en el archivo de Excel


# Cruce BDD Gestoras

In [13]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA GASTRICO'))

['BASE TRABAJO GESTION ONCOLOGICA_CANCER GASTRICO.xlsx',
 'Pacientes Gestión Oncológica 2024.xlsx',
 'Reporte diario casos nuevos',
 '~$BASE TRABAJO GESTION ONCOLOGICA_CANCER GASTRICO.xlsx']

In [14]:
archivo_excel = os.path.join(dir_actual, '..','CA GASTRICO', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER GASTRICO.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')

In [15]:
dataframe_2

,Problema Oncologico,Rut,DV,Nombre,Etapa,Nombre de la Garantía,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer Gástrico . {decreto nº 228},12497257,4,"ARGOMEDO SEPÚLVEDA, ROSA ELIZABETH",Seguimiento,NaN,Agendado Médico,2024-06-03,DR. JOSÉ LUIS GALINDO RIVERA,NaT,NaN,NaN,NaN,03-06-2024,DR. JOSÉ LUIS GALINDO RIVERA,2024-04-30,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,Cáncer Gástrico . {decreto nº 228},4042022,3,"TAPIA PARDO, IRENE CONCEPCIÓN",Seguimiento,NaN,Agendado Médico,2024-06-04,DR. CATERINA CONTRERAS BERTOLO,NaT,NaN,NaN,NaN,04-06-2024,DR. CATERINA CONTRERAS BERTOLO,2023-12-08,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,Cáncer Gástrico . {decreto nº 228},4806031,5,"MORALES FERNÁNDEZ, LUIS ALBERTO",Seguimiento,NaN,Agendado Médico,2024-06-05,DR. PEDRO PABLO MIGUEL SOTO GAJARDO,2024-01-11,Evaluación por oncología médica para quimioter...,NaN,940596924 // 962497008,05-06-2024,DR. PEDRO PABLO MIGUEL SOTO GAJARDO,2024-06-03,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,Cáncer Gástrico . {decreto nº 228},5229186,0,"MARTÍNEZ ASTORGA, GILBERTO ANTONIO",Seguimiento,NaN,Agendado Médico,2024-06-05,DR. ANDRÉS JAVIER MARAMBIO GRANIC,NaT,NaN,NaN,NaN,05-06-2024,DR. ANDRÉS JAVIER MARAMBIO GRANIC,2024-04-17,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,Cáncer Gástrico . {decreto nº 228},5745000,2,"TORRES TORRES, NELSON AURELIO",Confirmación Diagnóstica,NaN,Agendado Médico,2024-06-05,DR. ANDRÉS JAVIER MARAMBIO GRANIC,NaT,NaN,NaN,NaN,05-06-2024,DR. ANDRÉS JAVIER MARAMBIO GRANIC,2024-12-02,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,Cancer Gástrico,17878140,5,CARLOS ANDRÉS MARTÍNEZ BARAHONA,NaN,NaN,NaN,NaT,NaN,NaT,NaN,['Desnutricion protcalorica severa no especifi...,-/-/-,NaN,NaN,2022-06-14,Médico MAGDALENA LORETO BELLO SALINAS Rx Tórax...
833,Cancer Gástrico,24927526,3,VERONICA MARILINA MEDRANDA FARIAS,NaN,NaN,NaN,NaT,NaN,NaT,NaN,['Tumor maligno del estomago'],942478270/967572462/-,NaN,NaN,NaT,NaN
834,Cáncer Gástrico . {decreto nº 228},49946959,4,"LIRA TOVAR, ROSIRI VERONICA",NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
835,Cáncer Gástrico . {decreto nº 228},13954944,9,"SEPÚLVEDA CARRASCO, EUGENIO RAFAEL",Ingreso,NaN,Priorizacion CUD,NaT,NaN,NaT,NaN,- / - / / / 83206102 / 3224782 / -/./#ND/-,NaN,NaN,NaN,NaT,NaN


In [16]:

dataframe = resultado_filtrado_2.copy()

# Obtener la ruta del directorio actual del script
#dir_actual = os.path.dirname(__file__)
dir_actual = os.getcwd()


archivo_excel = os.path.join(dir_actual, '..','CA GASTRICO', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER GASTRICO.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


# Obtener la cantidad de filas que no son nulas en la columna 'mi_columna'
cantidad_filas_no_nulas = dataframe_2['Rut'].count()



print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
def quitar_ultimos_dos(elemento):
    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
for index, fila in filas_dv_nan.iterrows():
    rut = str(fila['RUT'])
    if True:
      dv = rut[-1]  # Último dígito del Rut
      dataframe_2.at[index, 'DV'] = dv
      dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut
      
dataframe_2['RUT'] = dataframe_2['RUT'].astype(float)
dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)


Cantidad de pacientes en la base 837


In [17]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Nombre de la Garantía,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer Gástrico . {decreto nº 228},12497257,4,"ARGOMEDO SEPÚLVEDA, ROSA ELIZABETH",Seguimiento,NaN,Agendado Médico,2024-06-03,DR. JOSÉ LUIS GALINDO RIVERA,NaT,NaN,NaN,NaN,03-06-2024,DR. JOSÉ LUIS GALINDO RIVERA,2024-04-30,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,Cáncer Gástrico . {decreto nº 228},4042022,3,"TAPIA PARDO, IRENE CONCEPCIÓN",Seguimiento,NaN,Agendado Médico,2024-06-04,DR. CATERINA CONTRERAS BERTOLO,NaT,NaN,NaN,NaN,04-06-2024,DR. CATERINA CONTRERAS BERTOLO,2023-12-08,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,Cáncer Gástrico . {decreto nº 228},4806031,5,"MORALES FERNÁNDEZ, LUIS ALBERTO",Seguimiento,NaN,Agendado Médico,2024-06-05,DR. PEDRO PABLO MIGUEL SOTO GAJARDO,2024-01-11,Evaluación por oncología médica para quimioter...,NaN,940596924 // 962497008,05-06-2024,DR. PEDRO PABLO MIGUEL SOTO GAJARDO,2024-06-03,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,Cáncer Gástrico . {decreto nº 228},5229186,0,"MARTÍNEZ ASTORGA, GILBERTO ANTONIO",Seguimiento,NaN,Agendado Médico,2024-06-05,DR. ANDRÉS JAVIER MARAMBIO GRANIC,NaT,NaN,NaN,NaN,05-06-2024,DR. ANDRÉS JAVIER MARAMBIO GRANIC,2024-04-17,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,Cáncer Gástrico . {decreto nº 228},5745000,2,"TORRES TORRES, NELSON AURELIO",Confirmación Diagnóstica,NaN,Agendado Médico,2024-06-05,DR. ANDRÉS JAVIER MARAMBIO GRANIC,NaT,NaN,NaN,NaN,05-06-2024,DR. ANDRÉS JAVIER MARAMBIO GRANIC,2024-12-02,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,Cancer Gástrico,17878140,5,CARLOS ANDRÉS MARTÍNEZ BARAHONA,NaN,NaN,NaN,NaT,NaN,NaT,NaN,['Desnutricion protcalorica severa no especifi...,-/-/-,NaN,NaN,2022-06-14,Médico MAGDALENA LORETO BELLO SALINAS Rx Tórax...
833,Cancer Gástrico,24927526,3,VERONICA MARILINA MEDRANDA FARIAS,NaN,NaN,NaN,NaT,NaN,NaT,NaN,['Tumor maligno del estomago'],942478270/967572462/-,NaN,NaN,NaT,NaN
834,Cáncer Gástrico . {decreto nº 228},49946959,4,"LIRA TOVAR, ROSIRI VERONICA",NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
835,Cáncer Gástrico . {decreto nº 228},13954944,9,"SEPÚLVEDA CARRASCO, EUGENIO RAFAEL",Ingreso,NaN,Priorizacion CUD,NaT,NaN,NaT,NaN,- / - / / / 83206102 / 3224782 / -/./#ND/-,NaN,NaN,NaN,NaT,NaN


# Merge Total pacientes vs Gestoras

In [18]:
# Ejemplo de creación de DataFrame (puedes cargar tus propios DataFrames)
df1 = dataframe.copy()
df2 = dataframe_2.copy()

# Agrega una columna para identificar el origen de cada RUT
df1['_df'] = 'df1'
df2['_df'] = 'df2'

# Convertir la columna 'RUT' en ambos DataFrames al mismo tipo de datos
df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)

#print(f"df_1: {df1['RUT'] }")
#print(f"df_2: {df2['RUT'] }")

# Realiza un merge de los dos DataFrames en la columna 'RUT' y conserva solo las filas que aparecen en 'left_only'
df_merge = pd.merge(df1, df2, on='RUT', how='outer', indicator=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Nombre del Paciente_x'].fillna(df_merge['Nombre del Paciente_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Nombre del Paciente_x': 'Nombre del Paciente'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Nombre del Paciente_y'], inplace=True)

# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Problema Oncologico_x'].fillna(df_merge['Problema Oncologico_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Problema Oncologico_x': 'Problema Oncologico'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Problema Oncologico_y'], inplace=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['DV_x'].fillna(df_merge['DV_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'DV_x': 'DV'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['DV_y'], inplace=True)

df_ca_tiroide = df_merge[df_merge['_merge']=='left_only']

# Reemplazar los valores NaN por '-'
df_ca_tiroide['N° Telefono 1'] = df_ca_tiroide['N° Telefono 1'].fillna('-')
df_ca_tiroide['N° Telefono 2'] = df_ca_tiroide['N° Telefono 2'].fillna('-')
df_ca_tiroide['N° Telefono 3'] = df_ca_tiroide['N° Telefono 3'].fillna('-')

def combinar_telefonos(row):
    return '/'.join(filter(None, [str(row['N° Telefono 1']), str(row['N° Telefono 2']), str(row['N° Telefono 3'])]))

# Aplicar la función a lo largo de las filas del DataFrame para crear la columna 'Telefono Consolidado'
df_ca_tiroide_telefono = df_ca_tiroide[['N° Telefono 1','N° Telefono 2','N° Telefono 3']]
df_ca_tiroide['Telefono Consolidado2'] = df_ca_tiroide_telefono.apply(combinar_telefonos, axis=1)

# Eliminar las columnas originales de los números de teléfono si ya no son necesarias
df_ca_tiroide.drop(columns=['N° Telefono 1', 'N° Telefono 2', 'N° Telefono 3'], inplace=True)

df_ca_tiroide['Etapa'] = np.nan

df_ca_tiroide['Estado'] = np.nan

df_ca_tiroide['Fecha Proximo Hito'] = np.nan

df_ca_tiroide['Observacion'] = np.nan

df_ca_tiroide['Medico Responsable'] = np.nan

df_ca_tiroide['Fecha Ultima atencion'] = np.nan

df_ca_tiroide['Profesional Ultima atencion'] = np.nan

df_ca_tiroide['MEDICO'] = np.nan

df_ca_tiroide['CITAS FUTURAS'] = np.nan

#df_ca_tiroide = df_ca_tiroide[['Problema Oncologico', 'RUT', 'DV', 'Nombre del Paciente','Etapa','Nombre garantia','Estado','Fecha Proximo Hito','Observacion','Medico Responsable','Fecha Ultima atencion', 'Profesional Ultima atencion','DIAGNOSTICOS', 'Telefono Consolidado2']]
df_ca_tiroide = df_ca_tiroide[['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Observacion',
 'Medico Responsable',
 'DIAGNOSTICOS',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']]

In [19]:
df_merge

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,...,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion,_df_y,_merge
0,Cancer Gástrico,10045349,5,MARY LUZ JÁUREGUI BARRERA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,['Tumor maligno del estomago'],-/-/-,NaN,NaN,2022-10-10,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ TAC ...,df2,right_only
1,Cancer Gástrico,10061635,1,PATRICIA VERÓNICA CONCHA ROCA,56959470836,56992075987,984896282.0,Confirmación Diagnóstica,NaN,[Tumor maligno del estomago],...,NaT,8-5-24: Paciente es controlada el 14-3-24 por ...,['Tumor maligno del estomago'],59470836/92075987/-,NaN,NaN,2024-10-02,Médico JOSÉ LUIS GALINDO RIVERA Interconsulta ...,df2,both
2,Cancer Gástrico,10122414,7,Olga Del Carmen Balladares Diaz,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,['Tumor de comportamiento incierto o desconoci...,-/-/-,NaN,NaN,NaT,NaN,df2,right_only
3,Cáncer Gástrico . {decreto nº 228},10123510,6,"GUERRA ORTIZ, PAULINA DEL ROSARIO",NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,15-5-24: Paciente evaluado por Dr. Galindo 13-...,NaN,#ND/#ND/-,NaN,NaN,NaT,NaN,df2,right_only
4,Cáncer Gástrico . {decreto nº 228},10126888,8,"BASUALTO CUEVAS, JUAN JOSÉ",NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,2023-09-20,Médico WENCESLAO JOSÉ ZEGERS VIAL TAC de Tórax...,df2,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,Cancer Gástrico,9794170,K,JOSÉ HERIBERTO VARGAS VALENZUELA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,['Tumor maligno del estomago'],953853965/977366699/998948630,NaN,NaN,2024-10-02,Médico ALVARO LUIS VILLAMAR ORMAZA TAC de Abdo...,df2,right_only
837,Cancer Gástrico,9855867,5,Carlos Mario Valenzuela Duarte,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,"['Tumor maligno del estomago', 'DOLOR ABDOMINA...",987779751/-/-,NaN,NaN,2022-09-03,Médico JOSÉ LUIS GALINDO RIVERA Derivacion des...,df2,right_only
838,Cancer Gástrico,9859350,0,MARCO ANTONIO MARAMBIO CASTRO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,['Tumor maligno del estomago'],-/-/-,NaN,NaN,NaT,NaN,df2,right_only
839,Cancer Gástrico,9879781,5,AMADOR ANTONIO MARTÍNEZ OLIVARES,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,13-5-24: Paciente no asistió a control con esp...,"['Cáncer Gástrico', 'Tumor maligno de la curva...",NaN,NaN,NaN,NaT,NaN,df2,right_only


In [20]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Nombre de la Garantía,Estado,Fecha Próximo Hito,Observacion,Medico Responsable,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
14,Cancer Gástrico,10331757,6,LORENA LAURA MUÑOZ CHAMORRO,NaN,NaN,NaN,NaT,NaN,NaN,"[Tumor maligno del estomago, Otras colitis y g...",0946833226/-/-,NaN,NaN,NaN,NaN
434,Cancer Gástrico,5519773,3,OLIVIA JUANA ZAPATA ABURTO,NaN,NaN,NaN,NaT,NaN,NaN,[Tumor maligno del estomago],41302755/22817079/-,NaN,NaN,NaN,NaN
630,Cancer Gástrico,6868187,1,ADOLFO JAVIER VARGAS ORREGO,NaN,NaN,NaN,NaT,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,990091473/-/-,NaN,NaN,NaN,NaN
780,Cancer Gástrico,8625490,5,MARLY ANGÉLICA CÁRDENAS MIRANDA,NaN,NaN,NaN,NaT,NaN,NaN,"[Nodulo tiroideo solitario no toxico, Tumor ma...",9964808077/945026901/996214387.0,NaN,NaN,NaN,NaN


In [21]:

df_ca_tiroide['Fecha de Comité'] = np.nan


In [22]:
lista = ['Problema Oncologico', 'RUT','DV','Nombre del Paciente','Etapa','Estado','Fecha Próximo Hito','Medico Responsable','Fecha de Comité','Observacion','DIAGNOSTICOS','Telefono Consolidado2','CITAS FUTURAS','MEDICO','Fecha Ultima atencion','Profesional Ultima atencion']

In [23]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Nombre de la Garantía,Estado,Fecha Próximo Hito,Observacion,Medico Responsable,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion,Fecha de Comité
14,Cancer Gástrico,10331757,6,LORENA LAURA MUÑOZ CHAMORRO,NaN,NaN,NaN,NaT,NaN,NaN,"[Tumor maligno del estomago, Otras colitis y g...",0946833226/-/-,NaN,NaN,NaN,NaN,NaN
434,Cancer Gástrico,5519773,3,OLIVIA JUANA ZAPATA ABURTO,NaN,NaN,NaN,NaT,NaN,NaN,[Tumor maligno del estomago],41302755/22817079/-,NaN,NaN,NaN,NaN,NaN
630,Cancer Gástrico,6868187,1,ADOLFO JAVIER VARGAS ORREGO,NaN,NaN,NaN,NaT,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,990091473/-/-,NaN,NaN,NaN,NaN,NaN
780,Cancer Gástrico,8625490,5,MARLY ANGÉLICA CÁRDENAS MIRANDA,NaN,NaN,NaN,NaT,NaN,NaN,"[Nodulo tiroideo solitario no toxico, Tumor ma...",9964808077/945026901/996214387.0,NaN,NaN,NaN,NaN,NaN


In [24]:
print(f'Cantidad de pacientes nuevos encontrados: {df_ca_tiroide.shape[0]}')

Cantidad de pacientes nuevos encontrados: 4


# Guardar estadística de llegada

In [ ]:
fecha_hoy = date.today()
data = {'Problema Oncologico': ['Cancer Gástrico'], 'Fecha Medición': [fecha_hoy], 'Cantidad de llegada': [df_ca_tiroide.shape[0]], 'Cantidad de pacientes base': [cantidad_filas_no_nulas]}
estadisticas_llegadas = pd.DataFrame(data)

nombre_archivo = f'Estadisticas_llegadas.xlsx'
archivo_excel = os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}')

df_existente = pd.read_excel(archivo_excel)
df_existente['Fecha Medición'] = pd.to_datetime(df_existente['Fecha Medición'])


wb = Workbook()
ws = wb.active
df_final = pd.concat([df_existente,estadisticas_llegadas], ignore_index=True)


for r in dataframe_to_rows(df_final, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'Estadisticas_llegadas.xlsx'
wb.save(os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}'))


# Agregar Fallecidos

In [ ]:
dir_actual = os.getcwd()
dir_gestoras = os.path.join(dir_actual,'..')

In [ ]:
ruta_del_archivo_2 = os.path.join(dir_gestoras, 'FALLECIDOS HLF CORTE 30_04_2024.xlsx')


In [ ]:
# Carga el archivo XLSX en un DataFrame
fallecidos = pd.read_excel(ruta_del_archivo_2)

In [ ]:
fallecidos_rut = pd.DataFrame({'RUT': fallecidos['RUD']})

In [ ]:
import pandas as pd

# Suponiendo que tus dataframes se llaman df_consolidado y df_fallecidos


# Combinamos los dataframes en función de la columna 'RUT'


fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)  # Convertir la columna a tipo string
fallecidos_rut['RUT'] = fallecidos_rut['RUT'].replace(['-', '\.'], '', regex=True) 

fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)


In [ ]:
df_pacientes_nuevos = df_ca_tiroide[lista].copy()

In [ ]:
df_pacientes_nuevos

In [ ]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Estado'] = 'Fallecido'


In [ ]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Etapa'] = 'Alta'


In [ ]:
df_pacientes_nuevos = df_pacientes_nuevos[~(df_pacientes_nuevos['RUT']=='nan')]

# Guardar dataframe

In [25]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA GASTRICO','Reporte diario casos nuevos'))

[]

In [26]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_Pacientes_nuevos_ca_gastrico.xlsx'  # Nombre del archivo de Excel
df_pacientes_nuevos.to_excel(os.path.join(dir_actual, '..','CA GASTRICO','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 

# Convertir la columna 'RUT' en DataFrame_2 a tipo object si es necesario


df_concatenated_2 = pd.concat([dataframe_2,df_pacientes_nuevos])

#print(list[df_concatenated_2.columns])




df_concatenated_2['RUT'] = df_concatenated_2['RUT'].astype(str)
resultado['RUT'] = resultado['RUT'].astype(str)
# Fusionar DataFrame_2 con el DataFrame de resultado utilizando la columna 'RUT' como clave de fusión

dataframe_final = df_concatenated_2.merge(resultado, on='RUT', how='left')

#print(list[dataframe_final.columns])


dataframe_final_2 = crear_dataframe_ultima_atencion(dataframe_final)

In [27]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,12497257,2024-04-30 09:55:00,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,4042022,2023-12-08 11:18:00,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,4806031,2024-06-03 10:38:00,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,5229186,2024-04-17 09:37:00,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,5745000,2024-12-02 19:34:00,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...
836,4720639,2024-10-05 14:33:00,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
837,10331757,NaT,NaN
838,5519773,2024-05-15 14:04:00,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
839,6868187,2024-08-05 13:48:00,Médico RUBÉN DARÍO ÁVILA SEPÚLVEDA Interconsul...


In [28]:
# Filtrar las filas donde 'Fecha Ultima atencion' no sea NaN
df_filtrado = dataframe_final_2[dataframe_final_2['Fecha Ultima atencion'].notna()]

# Mostrar el DataFrame filtrado
#print(df_filtrado)

In [29]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,12497257,2024-04-30 09:55:00,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,4042022,2023-12-08 11:18:00,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,4806031,2024-06-03 10:38:00,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,5229186,2024-04-17 09:37:00,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,5745000,2024-12-02 19:34:00,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...
836,4720639,2024-10-05 14:33:00,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
837,10331757,NaT,NaN
838,5519773,2024-05-15 14:04:00,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
839,6868187,2024-08-05 13:48:00,Médico RUBÉN DARÍO ÁVILA SEPÚLVEDA Interconsul...


In [30]:
dataframe_final_2 = dataframe_final_2.dropna()

In [31]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,12497257,2024-04-30 09:55:00,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,4042022,2023-12-08 11:18:00,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,4806031,2024-06-03 10:38:00,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,5229186,2024-04-17 09:37:00,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,5745000,2024-12-02 19:34:00,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...
832,17878140,2022-06-14 10:10:00,Médico MAGDALENA LORETO BELLO SALINAS Rx Tórax...
836,4720639,2024-10-05 14:33:00,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
838,5519773,2024-05-15 14:04:00,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
839,6868187,2024-08-05 13:48:00,Médico RUBÉN DARÍO ÁVILA SEPÚLVEDA Interconsul...


In [32]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%Y-%m-%d')


In [33]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%d-%m-%Y')


In [34]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,12497257,30-04-2024,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,4042022,08-12-2023,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,4806031,03-06-2024,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,5229186,17-04-2024,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,5745000,02-12-2024,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...
832,17878140,14-06-2022,Médico MAGDALENA LORETO BELLO SALINAS Rx Tórax...
836,4720639,05-10-2024,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
838,5519773,15-05-2024,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
839,6868187,05-08-2024,Médico RUBÉN DARÍO ÁVILA SEPÚLVEDA Interconsul...


In [35]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'],format='%d-%m-%Y')

In [36]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,12497257,2024-04-30,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,4042022,2023-12-08,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,4806031,2024-06-03,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,5229186,2024-04-17,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,5745000,2024-12-02,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...
832,17878140,2022-06-14,Médico MAGDALENA LORETO BELLO SALINAS Rx Tórax...
836,4720639,2024-10-05,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
838,5519773,2024-05-15,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
839,6868187,2024-08-05,Médico RUBÉN DARÍO ÁVILA SEPÚLVEDA Interconsul...


In [37]:
import pandas
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [38]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(dataframe_final_2, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_gastrico.xlsx' 
wb.save(os.path.join(dir_actual, '..','CA GASTRICO',f'{nombre_archivo}'))

## Guardar dataframe ultima atención 

In [39]:
#nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_gastrico.xlsx'  # Nombre del archivo de Excel
#dataframe_final_2.to_excel(os.path.join(dir_actual, '..','CA GASTRICO',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 


In [40]:
df1 = df_concatenated_2.copy()
df2 = dataframe_final_2.copy()

df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)


# Realizar la fusión utilizando la columna 'RUT'
df_merged = pd.merge(df1, df2, on='RUT', how='left')



In [41]:
df2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,12497257,2024-04-30,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,4042022,2023-12-08,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,4806031,2024-06-03,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,5229186,2024-04-17,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,5745000,2024-12-02,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...
832,17878140,2022-06-14,Médico MAGDALENA LORETO BELLO SALINAS Rx Tórax...
836,4720639,2024-10-05,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
838,5519773,2024-05-15,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
839,6868187,2024-08-05,Médico RUBÉN DARÍO ÁVILA SEPÚLVEDA Interconsul...


In [42]:
len(df1['RUT'].unique())

841

In [43]:
len(df2['RUT'].unique())

454

In [44]:
# Renombrar columnas para evitar colisiones
df_merged.rename(columns={'Fecha Ultima atencion_y': 'Fecha Ultima atencion', 'Profesional Ultima atencion_y': 'Profesional Ultima atencion'}, inplace=True)

# Eliminar columnas duplicadas
df_merged.drop(['Fecha Ultima atencion_x', 'Profesional Ultima atencion_x'], axis=1, inplace=True)



In [45]:
df_merged[df_merged.duplicated(subset=['RUT'], keep=False)]

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Nombre de la Garantía,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion


In [46]:
df_merged = df_merged.drop_duplicates(subset=['RUT'], keep='first')


In [47]:
df_merged

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Nombre de la Garantía,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer Gástrico . {decreto nº 228},12497257,4,"ARGOMEDO SEPÚLVEDA, ROSA ELIZABETH",Seguimiento,NaN,Agendado Médico,2024-06-03,DR. JOSÉ LUIS GALINDO RIVERA,NaT,NaN,NaN,NaN,03-06-2024,DR. JOSÉ LUIS GALINDO RIVERA,NaN,NaN,2024-04-30,Médico JOSÉ LUIS GALINDO RIVERA Control - Cons...
1,Cáncer Gástrico . {decreto nº 228},4042022,3,"TAPIA PARDO, IRENE CONCEPCIÓN",Seguimiento,NaN,Agendado Médico,2024-06-04,DR. CATERINA CONTRERAS BERTOLO,NaT,NaN,NaN,NaN,04-06-2024,DR. CATERINA CONTRERAS BERTOLO,NaN,NaN,2023-12-08,Médico CATERINA CONTRERAS BERTOLO TAC de Tórax...
2,Cáncer Gástrico . {decreto nº 228},4806031,5,"MORALES FERNÁNDEZ, LUIS ALBERTO",Seguimiento,NaN,Agendado Médico,2024-06-05,DR. PEDRO PABLO MIGUEL SOTO GAJARDO,2024-01-11,Evaluación por oncología médica para quimioter...,NaN,940596924 // 962497008,05-06-2024,DR. PEDRO PABLO MIGUEL SOTO GAJARDO,NaN,NaN,2024-06-03,Médico PEDRO PABLO MIGUEL SOTO GAJARDO Control...
3,Cáncer Gástrico . {decreto nº 228},5229186,0,"MARTÍNEZ ASTORGA, GILBERTO ANTONIO",Seguimiento,NaN,Agendado Médico,2024-06-05,DR. ANDRÉS JAVIER MARAMBIO GRANIC,NaT,NaN,NaN,NaN,05-06-2024,DR. ANDRÉS JAVIER MARAMBIO GRANIC,NaN,NaN,2024-04-17,Médico ANDRÉS JAVIER MARAMBIO GRANIC Control -...
4,Cáncer Gástrico . {decreto nº 228},5745000,2,"TORRES TORRES, NELSON AURELIO",Confirmación Diagnóstica,NaN,Agendado Médico,2024-06-05,DR. ANDRÉS JAVIER MARAMBIO GRANIC,NaT,NaN,NaN,NaN,05-06-2024,DR. ANDRÉS JAVIER MARAMBIO GRANIC,NaN,NaN,2024-12-02,Médico GERALDINE VALENTINA CORTÉS FAÚNDEZ Angi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,Cancer Gástrico,4720639,1,MARÍA LUCRECIA PIZARRO VERGARA,NaN,NaN,NaN,NaT,NaN,NaT,"['Otras anemias', 'Tumor maligno del estomago']",88210191/99956967/-,NaN,NaN,NaN,NaN,NaN,2024-10-05,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
837,Cancer Gástrico,10331757,6,LORENA LAURA MUÑOZ CHAMORRO,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno del estomago, Otras colitis y g...",0946833226/-/-,NaT,NaN
838,Cancer Gástrico,5519773,3,OLIVIA JUANA ZAPATA ABURTO,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,[Tumor maligno del estomago],41302755/22817079/-,2024-05-15,Médico MARIA GABRIELA CAMPOVERDE BRAVO Derivac...
839,Cancer Gástrico,6868187,1,ADOLFO JAVIER VARGAS ORREGO,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,[Tumor de comportamiento incierto o desconocid...,990091473/-/-,2024-08-05,Médico RUBÉN DARÍO ÁVILA SEPÚLVEDA Interconsul...


# Crear Citas futuras

In [48]:

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)



ruta_del_archivo = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','AgendasFuturas', f'{today.year}{month_formatted}{day_formatted}_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)

In [49]:
# Mover los nombres de las columnas una posición hacia la izquierda
new_columns = list(dataframe_fu.columns)
new_columns = new_columns[1:] + [new_columns[0]]
dataframe_fu.columns = new_columns


In [51]:

dataframe_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1715889412635310311,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco de partes blandas cadera der,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715356221635300908,Enfermera/TENS AD Oriente,Hospitalización Domiciliaria MQ,21-05-2024,800,Agendada,AD Profesional / Tens Básica,CEX,N,N,NaN,...,988035938,979262107,NaN,NaN,El Tamarindo,La Florida,988035938,979262107,NaN,NaN
1715889364576310298,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889364576310298,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889060263310209,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,VIC MACKENNA,La Florida,987551087,986127125,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715270935281300910,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1100,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,921712085,NaN,NaN,NaN,juan bagynka,La Florida,921712085,NaN,NaN,NaN
1713451086016300967,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1000,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,922269900,922269900,NaN,NaN,LAS ACACIAS,La Florida,922269900,922269900,NaN,NaN
1713451086016300967,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1000,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,922269900,922269900,NaN,NaN,LAS ACACIAS,La Florida,922269900,922269900,NaN,NaN
1715610771664310021,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1140,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0937564768,NaN,NaN,NaN,serrano,La Florida,0937564768,NaN,NaN,NaN


In [52]:
# Eliminar duplicados por índice y mantener el primero
df_sin_duplicados = dataframe_fu.drop_duplicates(keep='first')


In [53]:
df_sin_duplicados['ESTADO_CITA']

1715889412635310311    Agendada
1715356221635300908    Agendada
1715889364576310298    Agendada
1715889060263310209    Agendada
1713554280650310362    Agendada
                         ...   
1714662664397310870    Agendada
1713453317365300554    Agendada
1715270935281300910    Agendada
1713451086016300967    Agendada
1715610771664310021    Agendada
Name: ESTADO_CITA, Length: 28125, dtype: object

In [54]:
df_sin_duplicados['ESTADO_CITA'].unique()

array(['Agendada', 'Anulada', 'En buzón de reasignación'], dtype=object)

In [55]:
# Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
endocrinologia_df_fu = df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA'].isin(['Cirugía Abdominal Adulto','Medicina Interna','Imagenología AMB'])]
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
endocrinologia_df_fu = endocrinologia_df_fu[~endocrinologia_df_fu['ESTADO_CITA'].isin(['Anulada', 'En buzón de reasignación'])]

In [56]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1715889412635310311,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco de partes blandas cadera der,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889364576310298,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889060263310209,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,VIC MACKENNA,La Florida,987551087,986127125,NaN,NaN
1716047834547300440,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,TC de Abdomen y Pelvis con MC,CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,...,56962841903,56994512452,NaN,NaN,rucalin,La Florida,56962841903,56994512452,NaN,NaN
1716047858150300443,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,"TAC de Tórax, sin MC",CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,...,56962841903,56994512452,NaN,NaN,rucalin,La Florida,56962841903,56994512452,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715704743898300503,Sala I-4,Imagenología AMB,29-07-2024,1100,Agendada,Rx Antebrazo Izq F-L,CEX,N,N,NaN,...,0940757310,22938680,NaN,NaN,Pje 4,La Florida,0940757310,22938680,NaN,NaN
1715719328671300942,Sala I-4,Imagenología AMB,30-07-2024,1000,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,56963132377,979506748,995202400,NaN,Los Papiros,La Florida,56963132377,979506748,995202400,NaN
1712323640596910122,Sala I-4,Imagenología AMB,04-10-2024,840,Agendada,Rx Cadera Der Crosstable,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN
1712323464883910069,Sala I-4,Imagenología AMB,04-10-2024,830,Agendada,Rx Pelvis AP para evaluación de Prótesis,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN


In [57]:
# Convertir la columna de fecha a formato datetime
#endocrinologia_df_fu['CITAFECH'] = pd.to_datetime(endocrinologia_df_fu['CITAFECH'])

# Formatear la columna de fecha en el formato deseado
#endocrinologia_df_fu['CITAFECH'] = endocrinologia_df_fu['CITAFECH'].dt.strftime('%Y-%m-%d')

In [58]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1715889412635310311,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco de partes blandas cadera der,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889364576310298,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889060263310209,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,VIC MACKENNA,La Florida,987551087,986127125,NaN,NaN
1716047834547300440,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,TC de Abdomen y Pelvis con MC,CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,...,56962841903,56994512452,NaN,NaN,rucalin,La Florida,56962841903,56994512452,NaN,NaN
1716047858150300443,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,"TAC de Tórax, sin MC",CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,...,56962841903,56994512452,NaN,NaN,rucalin,La Florida,56962841903,56994512452,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715704743898300503,Sala I-4,Imagenología AMB,29-07-2024,1100,Agendada,Rx Antebrazo Izq F-L,CEX,N,N,NaN,...,0940757310,22938680,NaN,NaN,Pje 4,La Florida,0940757310,22938680,NaN,NaN
1715719328671300942,Sala I-4,Imagenología AMB,30-07-2024,1000,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,56963132377,979506748,995202400,NaN,Los Papiros,La Florida,56963132377,979506748,995202400,NaN
1712323640596910122,Sala I-4,Imagenología AMB,04-10-2024,840,Agendada,Rx Cadera Der Crosstable,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN
1712323464883910069,Sala I-4,Imagenología AMB,04-10-2024,830,Agendada,Rx Pelvis AP para evaluación de Prótesis,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN


In [59]:
endocrinologia_df_fu['DV'] = endocrinologia_df_fu['DOCUMENTO'].str[-1]

endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['DOCUMENTO'].str[:-1]

In [60]:
endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']=='RUN']

In [61]:
endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['RUT'].astype(int)

In [62]:
endocrinologia_df_fu = endocrinologia_df_fu[['RUT','NOMBRE_AGENDA','ESPECIALIDAD_AGENDA','CITAFECH','CITAHORA','ESTADO_CITA','DESCRIPCION','ORIGEN','GES_CITA','GES_SOL','PATOLOGIA_GES_SOL','PACIENTE','CITA_CODUTEN','DOCUMENTO']]

In [63]:
endocrinologia_df_fu

,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
1715889412635310311,28368794,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco de partes blandas cadera der,CEX,N,N,NaN,ALANA ESTEFANÍA GARCIA BERMUDEZ,AC00542379,283687945
1715889364576310298,28368794,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,ALANA ESTEFANÍA GARCIA BERMUDEZ,AC00542379,283687945
1715889060263310209,28368766,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,MAXIMILIANO ESTEBAN GARCIA BERMUDEZ,AC00542380,28368766K
1716047834547300440,7339477,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,TC de Abdomen y Pelvis con MC,CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,MERCEDES ROSA SEPÚLVEDA ARAVENA,AC00458224,73394775
1716047858150300443,7339477,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,"TAC de Tórax, sin MC",CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,MERCEDES ROSA SEPÚLVEDA ARAVENA,AC00458224,73394775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715704743898300503,26604801,Sala I-4,Imagenología AMB,29-07-2024,1100,Agendada,Rx Antebrazo Izq F-L,CEX,N,N,NaN,FRANCISCO ISAAC TORRES CORNEJO,AC00390929,266048017
1715719328671300942,9006844,Sala I-4,Imagenología AMB,30-07-2024,1000,Agendada,Rx Pelvis AP,CEX,N,N,NaN,ROSA INÉS DE LAS MER ZÚÑIGA SOTO,AC00058536,9006844K
1712323640596910122,22889659,Sala I-4,Imagenología AMB,04-10-2024,840,Agendada,Rx Cadera Der Crosstable,CEX,N,N,NaN,MATÍAS ANTONIO HUMERES MADARIAGA,AC00387114,228896594
1712323464883910069,22889659,Sala I-4,Imagenología AMB,04-10-2024,830,Agendada,Rx Pelvis AP para evaluación de Prótesis,CEX,N,N,NaN,MATÍAS ANTONIO HUMERES MADARIAGA,AC00387114,228896594


In [64]:
# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)


In [65]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(endocrinologia_df_fu, index= False, header = True):
    ws.append(r)

for cell in ws['D'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_gastrico.xlsx'
wb.save(os.path.join(dir_actual, '..','CA GASTRICO',f'{nombre_archivo}'))

In [66]:
#nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_gastrico.xlsx'  # Nombre del archivo de Excel
#endocrinologia_df_fu.to_excel(os.path.join(dir_actual, '..','CA GASTRICO',f'{nombre_archivo}'), index=False) 
